In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.random.seed(601)

# 1. 데이터 Load

In [2]:
train = pd.read_csv('data/train.csv', parse_dates=['pickup_datetime','dropoff_datetime'])
test = pd.read_csv('data/test.csv', parse_dates=['pickup_datetime'])
sample_submission = pd.read_csv('data/sample_submission.csv')

In [ ]:
train.head()
test.head()
sample_submission.head()

# 2. 데이터 전처리
## 2.1 EDA

In [ ]:
train.shape
test.shape
sample_submission.shape

In [ ]:
train.columns
test.columns
sample_submission.columns

In [ ]:
train.info()

In [ ]:
train.describe()

### 기간

In [ ]:
train.pickup_datetime.min(), train.pickup_datetime.max()
# 년도는 동일, 월, 일, 시간, 요일 추출 

test.pickup_datetime.min(), test.pickup_datetime.max()

In [ ]:
train.head()

In [11]:
# 운행 기간 Outlier 삭제 
train[(train.dropoff_datetime - train.pickup_datetime).dt.days > 1]
idx = train[(train.dropoff_datetime - train.pickup_datetime).dt.days > 1].index

train = train.drop(idx)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist
355003,id1864733,1,2016-01-05 00:19:42,2016-01-27 11:08:38,1,-73.789650,40.643559,-73.956810,40.773087,N,1939736,20.148692
680594,id0369307,1,2016-02-13 22:38:00,2016-03-08 15:57:38,2,-73.921677,40.735252,-73.984749,40.759979,N,2049578,5.982495
924150,id1325766,1,2016-01-05 06:14:15,2016-01-31 01:01:07,1,-73.983788,40.742325,-73.985489,40.727676,N,2227612,1.635130
978383,id0053347,1,2016-02-13 22:46:52,2016-03-25 18:18:14,1,-73.783905,40.648632,-73.978271,40.750202,N,3526282,19.900689


In [ ]:
train.shape # (1458644, 18) -> (1458640, 18)

### Target Date 시각화

In [ ]:
train.trip_duration.plot()

### Missing Value

In [ ]:
train.isnull().sum().sum()

### Value Counts

In [ ]:
def value_counts(df):
    for col in df.columns:
        print(col)
        print(train[col].value_counts())
        
value_counts(train[['vendor_id']])        

In [ ]:
pd.crosstab(train['vendor_id'], train['store_and_fwd_flag'], margins=True)

### trip_duration dropoff_datetime pickup_datetime 관계 확인 -> dropoff_datetime 빼고 학습?

In [3]:
from datetime import datetime, timedelta

In [ ]:
(train['trip_duration'] == (train['dropoff_datetime'] - train['pickup_datetime']).astype('timedelta64[s]')).value_counts()

## 시각화
### 지역: 경도/위도

#### x: 위도(latitude)
#### y: 경도(longitude) 

(pickup_latitude, pickup_longitude) -> (dropoff_latitude, dropoff_longitude)

In [4]:
from tqdm import tqdm_notebook
import time
InteractiveShell.ast_node_interactivity = "none"

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20, 10))
ax[0].set_title('pickup point')
ax[0].set_xlabel('latitude')
ax[0].set_ylabel('longitude')
ax[0].scatter(train.pickup_latitude, train.pickup_longitude, marker='o')
ax[1].set_title('dropoff point')
ax[1].set_xlabel('latitude')
ax[1].set_ylabel('longitude')
ax[1].scatter(train.dropoff_latitude, train.dropoff_longitude, marker='o')
plt.show()

outlier 가져갈것인가?

In [ ]:
start = time.time()  # 시작 시간 저장

total = tqdm_notebook(range(len(train)))

fig, ax = plt.subplots(1,1, figsize=(20, 10))

for i in total:
    plt.arrow(train.pickup_latitude[i], train.pickup_longitude[i]
            , (train.dropoff_latitude[i]-train.pickup_latitude[i])
            , (train.dropoff_longitude[i]-train.pickup_longitude[i]), head_width=0.01, head_length=0.01, fc='k', ec='k')
    
ax.set_title('start -> end arrow')
ax.set_xlim(32, 42)
ax.set_ylim(-90, -60)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

### 위도 경도로 거리 구하기 -> haversine

In [5]:
from haversine import haversine

In [6]:
train['dist'] = 0.0

start = time.time()  # 시작 시간 저장

total = tqdm_notebook(range(len(train)))

for i in total:
    train['dist'][i] = haversine((train.pickup_latitude[i], train.pickup_longitude[i]), (train.dropoff_latitude[i], train.dropoff_longitude[i]))
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


time : 1601.223747253418


In [7]:
test['dist'] = 0.0

start = time.time()  # 시작 시간 저장

total = tqdm_notebook(range(len(test)))

for i in total:
    test['dist'][i] = haversine((test.pickup_latitude[i], test.pickup_longitude[i]), (test.dropoff_latitude[i], test.dropoff_longitude[i]))
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


time : 196.38118934631348


In [8]:
InteractiveShell.ast_node_interactivity = "all"

순수하게 Load 해서 거리만 넣은 피클 

In [9]:
import pickle

with open("train.pkl", "wb") as f:
    pickle.dump(train, f)
    
with open("test.pkl", "wb") as f:
    pickle.dump(test, f)    

In [10]:
with open("train.pkl", "rb") as f:
    train = pickle.load(f)    
    
with open("test.pkl", "rb") as f:
    test = pickle.load(f)        

In [ ]:
train.head()
test.head()

### 시간: 픽업시간 (시간 -> 요일(평일/주말(공휴일)))

In [ ]:
InteractiveShell.ast_node_interactivity = "none"

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20, 10))

# 날짜별 분포
ymd = train.pickup_datetime.dt.strftime('%Y-%m-%d').value_counts().sort_index()
ax[0].set_title('date hist')
ax[0].set_xlabel('date')
ax[0].set_ylabel('count')
ax[0].plot(ymd)

# 평일/주말 분포 # (0:월, 1:화, 2:수, 3:목, 4:금, 5:토, 6:일)
wk = train.pickup_datetime.dt.weekday.value_counts().sort_index()
ax[1].set_title('weekday hist')
ax[1].set_xlabel('weekday')
ax[1].set_ylabel('count')
ax[1].plot(wk)


# 시간별 분포 
hr = train.pickup_datetime.dt.strftime('%H').value_counts().sort_index()
ax[2].set_title('hour hist')
ax[2].set_xlabel('hour')
ax[2].set_ylabel('count')
ax[2].plot(hr)

plt.show()

#### 1) date hist: 특정 날에 건수가 없는 날이 있음 제외 고려, 년/월/일/시 쪼개서 넣을것인가? 저 특이한 날은 제외하는게 학습에 도움이 되는가? 상관이 없는가? count 가 적으므로 dist 도 적어질테니까 없애는게 나을거 같은데... 
#### 2) weekday hist(0:월, 1:화, 2:수, 3:목, 4:금, 5:토, 6:일): 요일별 편차가 존재함
#### 3) hour hist: 새벽시간 건수가 적음 -> 시간별로 묶는 변수를 추가할까? 

In [ ]:
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 10))

# 시간별 탑승 건수
datetimeCnt = train.pickup_datetime.dt.strftime('%Y-%m-%d %H').value_counts().sort_index()

ax.set_title('datetime - count')
ax.set_xlabel('datetime')
ax.set_ylabel('count')
ax.plot(datetimeCnt)

plt.show()

In [ ]:
# 거리 분포
train.dist.plot()

In [ ]:
train['dist'].groupby(train.pickup_datetime.dt.strftime('%Y-%m-%d')).sum()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 10))

# 날자별 탑승 거리 
dateDist = train['dist'].groupby(train.pickup_datetime.dt.strftime('%Y-%m-%d')).sum()

ax.set_title('date - dist')
ax.set_xlabel('date')
ax.set_ylabel('dist')
ax.plot(dateDist)

plt.show()

In [ ]:
dateDist[dateDist == dateDist.min()]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 10))

# 날자별 탑승 거리 
dateDist_drop = train_drop['dist'].groupby(train_drop.pickup_datetime.dt.strftime('%Y-%m-%d')).sum()

ax.set_title('date - dist')
ax.set_xlabel('date')
ax.set_ylabel('dist')
ax.plot(dateDist_drop)

plt.show()

In [ ]:
dateDist_drop[dateDist_drop == dateDist_drop.min()]

In [ ]:
# '2016-01-24' 날짜 지워보기
idx = train_drop[(train_drop.pickup_datetime.dt.strftime('%Y-%m-%d') == '2016-01-24')].index

train_drop = train_drop.drop(idx)
train_drop

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 10))

# 날자별 탑승 거리 
dateDist_drop = train_drop['dist'].groupby(train_drop.pickup_datetime.dt.strftime('%Y-%m-%d')).sum()

ax.set_title('date - dist')
ax.set_xlabel('date')
ax.set_ylabel('dist')
ax.plot(dateDist_drop)

plt.show()

## 클러스터링
### 지역 클러스터: 경도/위도
-> 거리 
### 시간 클러스터: 픽업시간 시간 
-> 요일(평일/주말(공휴일)))

=> 클러스터링 결과로?
1) 클러스터로 묶어서 레코드를 추가할 수도 있고
2) 클러스터끼리 학습할 수도 있을것 같고

In [ ]:
InteractiveShell.ast_node_interactivity = "all"

In [12]:
# 년도는 동일, 월, 일, 시간, 요일 추출 

train['pickup_date'] = train.pickup_datetime.dt.strftime('%m%d')
train['pickup_day'] = train['pickup_datetime'].dt.day
train['pickup_hour'] = train['pickup_datetime'].dt.hour
train['pickup_weekday'] = train.pickup_datetime.dt.weekday

test['pickup_date'] = test.pickup_datetime.dt.strftime('%m%d')
test['pickup_day'] = test['pickup_datetime'].dt.day
test['pickup_hour'] = test['pickup_datetime'].dt.hour
test['pickup_weekday'] = test.pickup_datetime.dt.weekday

## 인코딩

In [ ]:
train.head()
test.head()

In [13]:
train = pd.get_dummies(train, columns=['vendor_id'], prefix='vendor_id')
test = pd.get_dummies(test, columns=['vendor_id'], prefix='vendor_id')

In [14]:
train = pd.get_dummies(train, columns=['store_and_fwd_flag'], prefix='store_and_fwd_flag')
test = pd.get_dummies(test, columns=['store_and_fwd_flag'], prefix='store_and_fwd_flag')

# 3. 모델학습

In [15]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [16]:
train['pickup_mm'] = train.pickup_datetime.dt.strftime('%m').astype('int')
train['pickup_dd'] = train.pickup_datetime.dt.strftime('%d').astype('int')

test['pickup_mm'] = test.pickup_datetime.dt.strftime('%m').astype('int')
test['pickup_dd'] = test.pickup_datetime.dt.strftime('%d').astype('int')

In [17]:
train.columns

Index(['id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'trip_duration', 'dist', 'pickup_date',
       'pickup_day', 'pickup_hour', 'pickup_weekday', 'vendor_id_1',
       'vendor_id_2', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y',
       'pickup_mm', 'pickup_dd'],
      dtype='object')

In [18]:
#train.columns

columns = [
           #'vendor_id'
            'vendor_id_1','vendor_id_2'
          ,'passenger_count'
           #, 'pickup_datetime'
           #,'pickup_date'
           ,'pickup_mm','pickup_dd'
           ,'pickup_day','pickup_hour','pickup_weekday'
           ,'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'
           ,'dist'
           ,'store_and_fwd_flag_N','store_and_fwd_flag_Y'
          ]
X_train = train[columns]
#y_train = train['trip_duration']
y_train = np.log1p(train['trip_duration'])

X_test = test[columns]

In [ ]:
X_train.head()
X_train.info()

In [19]:
li_reg = LinearRegression()
li_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
rf_reg = RandomForestRegressor(random_state=0, n_estimators=1000, n_jobs=-1)
rf_reg.fit(X_train, y_train)

MemoryError: could not allocate 117440512 bytes

In [21]:
xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

MemoryError: Unable to allocate 83.5 MiB for an array with shape (21879600,) and data type float32

In [ ]:
lgb_reg = LGBMRegressor()
lgb_reg.fit(X_train, y_train)

oneHot-store_and_fwd_flag-LinearRegression
oneHot-store_and_fwd_flag-RandomForestRegressor
oneHot-store_and_fwd_flag-XGBRegressor
oneHot-store_and_fwd_flag-LGBMRegressor

In [ ]:
y_pred_li = li_reg.predict(X_test)

In [ ]:
y_pred_rf = rf_reg.predict(X_test)

In [ ]:
y_pred_xgb = xgb_reg.predict(X_test)

In [ ]:
y_pred_lgb = lgb_reg.predict(X_test)

In [ ]:
#sample_submission['trip_duration'] = y_pred_li
sample_submission['trip_duration'] = np.expm1(y_pred_li)

In [ ]:
sample_submission.to_csv('./output/20200819-21.csv', index=False)

In [ ]:
#sample_submission['trip_duration'] = y_pred_rf
sample_submission['trip_duration'] = np.expm1(y_pred_rf)

In [ ]:
sample_submission.to_csv('./output/20200819-22.csv', index=False)

In [ ]:
#sample_submission['trip_duration'] = y_pred_xgb
sample_submission['trip_duration'] = np.expm1(y_pred_xgb)

In [ ]:
sample_submission.to_csv('./output/20200819-23.csv', index=False)

In [ ]:
#sample_submission['trip_duration'] = y_pred_lgb
sample_submission['trip_duration'] = np.expm1(y_pred_lgb)

In [ ]:
sample_submission.to_csv('./output/20200819-24.csv', index=False)